## Project 2 (Data Analysis) (WIP)
### Stack Overflow annual developer survey


1) At what companies do developers get paid the most?
2) How much does remote working matter to employees?
3) What's the most popular method of learning to code?
4) Are you more likely to get a job as a developer if you have a master's degree?


In [28]:
import pandas as pd
import re
from datetime import datetime
import requests


df = pd.read_csv("survey_results_public.csv")
df_schema = pd.read_csv("survey_results_schema.csv")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print(list(df))
print(list(df_schema))

['ResponseId', 'Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith', 'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith', 'AISearchWantToWorkWith', 'AIDevHaveWorkedWith', 'AIDevWantToWorkWith', 'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFr

In [29]:
# At what companies to developers get paid the most?

# Step 0) Clean the data set. Remove any rows that have NaN values in either the Currency, Industry or CompTotal columns. Change Currency column to currency codes.

df_cleaned = df[~(df['Currency'].isna() | df['Industry'].isna() | df['CompTotal'].isna())]
data_length = len(df_cleaned)

def extract_currency_code(entry):
    return re.split(r'[ \t]+', entry)[0]
    
df_cleaned['Currency_Code'] = df_cleaned['Currency'].apply(extract_currency_code)
#print(df_cleaned.head())



#print(df['Currency'])
#print(df[~df['Currency'].isna()])

#print(len(df['Currency']))
#print(len(df['Currency'].isnull()))

# We should first extract the "Industry" answers and comoute the mean sallary for each industry type. (The industry question is multiple choice)

#Industry_dict = {}
#print(df.head())
#num_entries = len(df)

#df_cleaned = df[~((df['Industry'].isna()) | df[''])]

#for i in range(num_entries):
    #

/tmp/ipykernel_158918/4008653894.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Currency_Code'] = df_cleaned['Currency'].apply(extract_currency_code)


In [32]:
# Step 1) Since we are dealing with different currencies let's first write a function that converts all currencies to USD.


# Function to get the historical exchange rate from Alpha Vantage API
def get_exchange_rate(api_key, target_currency='USD'):

    url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{target_currency}'
    response = requests.get(url)
    data = response.json()
    return data['conversion_rates']

rate_dict = get_exchange_rate('Your_API_Key_Here', 'USD') # get latest conversion USD conversion rates
    


# Define a function to convert amount to USD
def currency_conv(api_key, currency_code, amount):
    if currency_code == 'USD':
        return amount 
    rate = rate_dict[currency_code]
    return amount * rate



# Assume you have an API key
api_key = 'Your_API_Key_Here'

# Apply the conversion function to each row
df_cleaned['Comp_USD'] = df_cleaned.apply(lambda row: currency_conv(api_key, row['Currency_Code'], row['CompTotal']), axis=1)


{'result': 'error', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms-of-use': 'https://www.exchangerate-api.com/terms', 'error-type': 'quota-reached'}


In [ ]:
# What's the most popular method of learning to code?

In [ ]:
# Are you more likely to get a job as a developer if you have a master's degree?
pd.set_option('display.max_rows', None)

count_total_master = 0
count_dev_master = 0

df['EdLevel'] = df['EdLevel'].fillna('')
df['Employment'] = df['Employment'].fillna('')

for row in range(df.shape[0]):
    if 'Master' in df['EdLevel'][row]:
        count_total_master += 1
        if 'I am a developer' in df['MainBranch'][row]:
            count_dev_master += 1
print(count_dev_master/count_total_master)
    #print(df['EdLevel'][i],df['Employment'][i])

In [ ]:
import pandas as pd

def get_percentage_devs_with_masters(df):
    """
    Calculate the percentage of developers with a master's degree among those with a master's degree.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame.
        
    Returns:
        float: Percentage of developers with a master's degree.
    """
    pd.set_option('display.max_rows', None)

    # Fill NaN values in relevant columns
    df['EdLevel'] = df['EdLevel'].fillna('')
    df['Employment'] = df['Employment'].fillna('')

    # Filter rows where 'Master' is in 'EdLevel'
    master_condition = df['EdLevel'].str.contains('Master', case=False)
    df_master = df[master_condition]

    # Count total and developer master's degrees
    count_total_master = df_master.shape[0]
    count_dev_master = df_master[df_master['MainBranch'].str.contains('I am a developer')].shape[0]

    # Calculate and return the percentage
    percentage_dev_master = (count_dev_master / count_total_master) * 100
    return percentage_dev_master

# Example usage
result = get_percentage_devs_with_masters(df)
print(result)


In [ ]:
print(dir(df['EdLevel']))

In [ ]:
print(str(5))
x = 6
print(dir(str))